In [22]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import datetime
import os
from feature_engineering.tensor_features import develop_features, floating_conv
from sklearn.preprocessing import StandardScaler
from data_handler import LocalToLargeDataLoader
from path_finder import path_sorter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
data_loader = LocalToLargeDataLoader(print_progress=True)
parsed_data = data_loader.load_raw_data(path="../../resources")

Retrieving training data...


In [16]:
index_data = parsed_data.copy()
index_data.set_index("time", inplace=True)


In [ ]:
import loadBar


def fill_with_proximity(df):
    # Forward fill first
    df_ffill = df.ffill()
    
    # Backward fill next
    df_bfill = df.bfill()
    # Create a mask for original missing values
    mask = df.isna()

    # Create a new DataFrame to hold the results
    filled_df = df.copy()

    # Iterate over each missing value
    for i in range(len(df)):
        if mask.iloc[i, 0]:  # Check if the value is missing
            # Get the current index (timestamp)
            current_time = df.index[i]

            # Get the last known value index (timestamp)
            last_known_index = df_ffill.first_valid_index()
            if last_known_index is not None:
                distance_to_ffill = (current_time - last_known_index).total_seconds()  # distance to last known

            # Get the next known value index (timestamp)
            next_known_index = df_bfill.first_valid_index()
            if next_known_index is not None:
                distance_to_bfill = (next_known_index - current_time).total_seconds()  # distance to next known

            # Fill based on proximity
            if (last_known_index is not None and distance_to_ffill < distance_to_bfill) or next_known_index is None:
                filled_df.iloc[i] = df_ffill.iloc[i]
            else:
                filled_df.iloc[i] = df_bfill.iloc[i]

    return filled_df



def resampler(df, sorting_column, freq):
    unique_ids = df[sorting_column].unique()
    final_df = pd.DataFrame()
    partial_list = []

    for i in range(len(unique_ids)):
        loadBar.load_bar(len(unique_ids),i+1)
        resample_partial = df[df[sorting_column] == unique_ids[i]].resample(freq).last()
        resample_partial = fill_with_proximity(resample_partial)
        partial_list.append(resample_partial)

    for chunk in partial_list:
        final_df = pd.concat([final_df,chunk])
    
    return final_df

resampled_data_h = resampler(index_data, "vesselId", "h")
resampled_data_20min = resampler(index_data, "vesselId", "20min")

resampled_data_h.to_csv('resampled_data_h.csv')
resampled_data_20min.to_csv('resampled_data_20min.csv')


KeyboardInterrupt: 

In [17]:
#We have two resampled and therefore regular datasets, now we need to make our time series into a supervised problem.
#First I need to change navstat into a categorical feature:


# Define categories based on ranges or discrete values
pretty_20m = pd.read_csv("../../resources/resampled_data_20min.csv")

pretty_h = pd.read_csv("../../resources/resampled_data_h.csv")

navstat_unique = pretty_h["navstat"].unique()

pretty_20m["navstat"] = pd.Categorical(pretty_20m["navstat"], categories=navstat_unique, ordered=True)

pretty_h["navstat"] = pd.Categorical(pretty_h["navstat"], categories=navstat_unique, ordered=True)


# Let's make dummys 

pretty_h = pd.get_dummies(pretty_h, columns=["navstat"], drop_first=True)
pretty_20m = pd.get_dummies(pretty_20m, columns = ["navstat"], drop_first=True)

pretty_20m.set_index("time", inplace=True)
pretty_h.set_index("time", inplace=True)


In [18]:
print(pretty_h.head())
print(pretty_h.shape)

                       cog   sog  rot  heading  latitude  longitude  \
time                                                                  
2024-01-01 00:00:00  284.0   0.7  0.0     88.0 -34.74370   -57.8513   
2024-01-01 01:00:00   88.2  14.3  0.0     86.0 -35.16805   -56.5319   
2024-01-01 02:00:00   88.2  14.3  0.0     86.0 -35.16805   -56.5319   
2024-01-01 03:00:00   88.2  14.3  0.0     86.0 -35.16805   -56.5319   
2024-01-01 04:00:00   88.2  14.3  0.0     86.0 -35.16805   -56.5319   

                                     vesselId                    portId  \
time                                                                      
2024-01-01 00:00:00  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
2024-01-01 01:00:00  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
2024-01-01 02:00:00  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
2024-01-01 03:00:00  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
2024-01-01 04:00:00  61e9f3a8b937134a3c4bfdf7  61d37

In [19]:
#Make time series into supervised problem
def make_supervised(df, forecast_columns, sorting_column, input_window=1, output_window=1):
    """
    Converts a multivariate time series dataframe into a supervised learning problem.
    
    Parameters:
    df (pd.DataFrame): The original dataframe with time series data.
    forecast_columns (list): A list of column names to forecast.
    input_window (int): The number of past observations to use as features.
    output_window (int): The number of steps to forecast into the future.
    
    Returns:
    pd.DataFrame: A new dataframe with supervised learning format.
    """
    # Create a dataframe to hold the transformed features
    df_supervised = pd.DataFrame()

    #Put in a for loop here where you iterate over all IDs, to make sure things get correct
    unique_sorts = df[sorting_column].unique()
    
    #Iterate through all IDs
    for sorts in unique_sorts:
        sort_df = df[df[sorting_column] == sorts]

        #Iterate through all columns for input features
        for col in sort_df.columns: 
            for i in range(input_window, 0, -1):
                df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
            df_supervised[f"{col}_t"] = sort_df[col]

    # Create columns for forecast (target) with forward shift
        for col in forecast_columns:
            for j in range(output_window, 0, -1):
                df_supervised[f"{col}_t+{j}"] = sort_df[col].shift(-j)
        

    # Remove rows with NaN values caused by the shifting process
    df_supervised.dropna(inplace=True)
    
    return df_supervised

supervised_h = make_supervised(pretty_h, ["latitude", "longitude"],"vesselId" , 3, 2)

supervised_h.columns

C:\Users\Bruker\AppData\Local\Temp\ipykernel_10836\1265698822.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
C:\Users\Bruker\AppData\Local\Temp\ipykernel_10836\1265698822.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
C:\Users\Bruker\AppData\Local\Temp\ipykernel_10836\1265698822.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

Index(['cog_t-3', 'cog_t-2', 'cog_t-1', 'cog_t', 'sog_t-3', 'sog_t-2',
       'sog_t-1', 'sog_t', 'rot_t-3', 'rot_t-2',
       ...
       'navstat_13.0_t-1', 'navstat_13.0_t', 'navstat_9.0_t-3',
       'navstat_9.0_t-2', 'navstat_9.0_t-1', 'navstat_9.0_t', 'latitude_t+2',
       'latitude_t+1', 'longitude_t+2', 'longitude_t+1'],
      dtype='object', length=112)

In [ ]:
def sort_columns(df):
    
    # Extract suffixes and assign _t as _t0
    columns_with_suffix = []
    for col in df.columns:
        match = re.search(r"_t([+-]?\d*)$", col)
        # If there's no number after _t, treat it as _t0
        suffix = int(match.group(1)) if match.group(1) else 0
        columns_with_suffix.append((col, suffix))
    
    # Sort by suffix value (ascending)
    sorted_t_columns = [col for col, _ in sorted(columns_with_suffix, key=lambda x: x[1])]
    
    # Reorder dataframe columns
    return df[sorted_t_columns]

supervised_t=sort_columns(supervised_h)

In [34]:
for col in supervised_t.columns:
    print(col)

latitude_t+2
longitude_t+2
latitude_t+1
longitude_t+1
cog_t
sog_t
rot_t
heading_t
latitude_t
longitude_t
vesselId_t
portId_t
etaParsed_t
UN_LOCODE_t
ISO_t
portLongitude_t
portLatitude_t
navstat_5.0_t
navstat_1.0_t
navstat_8.0_t
navstat_2.0_t
navstat_3.0_t
navstat_15.0_t
navstat_4.0_t
navstat_14.0_t
navstat_11.0_t
navstat_12.0_t
navstat_6.0_t
navstat_7.0_t
navstat_13.0_t
navstat_9.0_t
cog_t-1
sog_t-1
rot_t-1
heading_t-1
latitude_t-1
longitude_t-1
vesselId_t-1
portId_t-1
etaParsed_t-1
UN_LOCODE_t-1
ISO_t-1
portLongitude_t-1
portLatitude_t-1
navstat_5.0_t-1
navstat_1.0_t-1
navstat_8.0_t-1
navstat_2.0_t-1
navstat_3.0_t-1
navstat_15.0_t-1
navstat_4.0_t-1
navstat_14.0_t-1
navstat_11.0_t-1
navstat_12.0_t-1
navstat_6.0_t-1
navstat_7.0_t-1
navstat_13.0_t-1
navstat_9.0_t-1
cog_t-2
sog_t-2
rot_t-2
heading_t-2
latitude_t-2
longitude_t-2
vesselId_t-2
portId_t-2
etaParsed_t-2
UN_LOCODE_t-2
ISO_t-2
portLongitude_t-2
portLatitude_t-2
navstat_5.0_t-2
navstat_1.0_t-2
navstat_8.0_t-2
navstat_2.0_t-2
navs